In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder

from sklearn.model_selection import train_test_split


import folium
import plotly.express as px

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.linear_model import LinearRegression


In [2]:
df=pd.read_csv("feature_eng_data.csv")

In [3]:
df.set_index('Index', inplace=True)

In [4]:
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [5]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,em_acount,country_id,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,1,0,29.0,0,35,0,87218.10,0,0,1
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,35548.74,0,0,1
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,122179.11,0,0,1
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,0,0,50.0,0,22,0,119775.54,0,1,0
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,1,0,50.0,1,23,0,96796.23,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,50.0,1,22,0,43912.17,0,0,1
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,26.0,1,23,0,23334.99,0,0,1
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,1,0,50.0,0,47,0,96796.23,0,0,1


In [6]:
df['diff_productos'] = df.groupby('pk_cid')['Cuentas'].diff()

In [7]:
df['diff_productos']

Index
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
5962919    0.0
5962920    0.0
5962921    0.0
5962922    0.0
5962923    NaN
Name: diff_productos, Length: 5936633, dtype: float64

In [8]:
df['compra'] = (df['diff_productos'] >= 1).astype(int)

In [9]:
df['compra'].value_counts()

compra
0    5851186
1      85447
Name: count, dtype: int64

In [10]:
#Comprobamos con un pk_cid que sabemso que tiene mas de un producto y vemos que detecta las compras correctamente

df[df['pk_cid'] == 16502]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra
Index,,,,,,,,,,,,,,,,,,,,,
2171490,16502,2018-09-28,2018-09-30,KHE,0.0,3,0,0,0,0,...,28.0,0,57,0,126765.57,0,0,0,NaN,0
2943633,16502,2018-10-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,1.0,1
2962972,16502,2018-11-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
3628237,16502,2018-12-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
4028330,16502,2019-01-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,1.0,1
4480638,16502,2019-02-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
4876038,16502,2019-03-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
5134320,16502,2019-04-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
5523395,16502,2019-05-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0


In [11]:
#Llenamos los NaN del primer mes con '0' ya que no tenemos con que comparar 

In [12]:
df['diff_productos'].fillna(value=0, inplace=True)


In [13]:
#Por tanto 'compra' sera nuestro target para la predisposicion de compra de los productos "CUENTAS"

In [14]:
df['entry_date'] = pd.to_datetime(df['entry_date'])
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [15]:
df['Antiguedad'] = (df['pk_partition'] - df['entry_date']) / pd.Timedelta(days=30)


In [16]:
df['Antiguedad'] = df['Antiguedad'].round().astype(int)

In [17]:
df['mes'] = df['pk_partition'].dt.to_period('M')


In [18]:
serie_temporal = df.groupby('mes')['compra'].sum()


In [19]:
serie_temporal

mes
2018-01       0
2018-02    3592
2018-03    3592
2018-04    3168
2018-05    3580
2018-06    3946
2018-07    3955
2018-08    6215
2018-09    5386
2018-10    7016
2018-11    6404
2018-12    6779
2019-01    5351
2019-02    7659
2019-03    6039
2019-04    6031
2019-05    6734
Freq: M, Name: compra, dtype: int32

In [20]:
serie_temporal = serie_temporal.drop(serie_temporal.index[0])

In [21]:
#Intentaremos predecir cuantos productos de CUENTAS venderemos el mes siguiente (JUNIO DE 2019)

df_regresion = serie_temporal.reset_index()
df_regresion['mes'] = df_regresion['mes'].astype(str)  # Convertimos el periodo a string para la regresión lineal

# Dividimos los datos en características (X) y la variable objetivo (y)
X = pd.to_numeric(df_regresion['mes'].str.replace('-', ''), errors='coerce').values.reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la regresión lineal
y = df_regresion['compra'].values

# Entrenamos el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X, y)

# Predecimos las compras del mes 06 del año 2019
mes_06_2019 = pd.to_numeric('2019-06'.replace('-', ''), errors='coerce').reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la predicción
prediccion_mes_06_2019 = modelo.predict(mes_06_2019)

print("Predicción de compras para el mes 06 del año 2019:", prediccion_mes_06_2019[0])

Predicción de compras para el mes 06 del año 2019: 6504.497877358459


In [32]:
#Predicción de beneficios

Beneficios = prediccion_mes_06_2019[0]*10
Beneficios

#Beneficios estimados para JUNIO 2019 de cproductos de CUENTAS = 65.044 € 

65044.97877358459

In [22]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra,Antiguedad,mes
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,35,0,87218.10,0,0,1,0.0,0,1,2018-01
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,23,0,35548.74,0,0,1,0.0,0,30,2018-01
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,23,0,122179.11,0,0,1,0.0,0,30,2018-01
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,22,0,119775.54,0,1,0,0.0,0,30,2018-01
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,23,0,96796.23,0,0,1,0.0,0,30,2018-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,22,0,43912.17,0,0,1,0.0,0,34,2019-05
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,23,0,23334.99,0,0,1,0.0,0,34,2019-05
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,47,0,96796.23,0,0,1,0.0,0,34,2019-05


In [23]:
df = df.drop(['entry_date', 'pk_partition','deceased','pk_cid','diff_productos'], axis=1)


In [24]:
df= pd.get_dummies(df, columns=['entry_channel'], drop_first=True)

In [25]:
X = df.drop('compra', axis=1)  # Features
y = df['compra']  # Target

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
print("Tamaño del conjunto de entrenamiento (X):", X_train.shape)
print("Tamaño del conjunto de prueba (X):", X_test.shape)
print("Tamaño del conjunto de entrenamiento (y):", y_train.shape)
print("Tamaño del conjunto de prueba (y):", y_test.shape)

Tamaño del conjunto de entrenamiento (X): (4749306, 39)
Tamaño del conjunto de prueba (X): (1187327, 39)
Tamaño del conjunto de entrenamiento (y): (4749306,)
Tamaño del conjunto de prueba (y): (1187327,)


In [28]:
def AUC(model, X, y):
    return roc_auc_score(y, model.predict_proba(X)[:, 1])

def cross_validation(model, X, y):
    Kfold_metric = cross_val_score(model, X, y, cv=4, scoring='roc_auc')
    return Kfold_metric.mean()

In [29]:
label_ = []
metric_train_ = []
metric_valida_ = []

In [30]:
def evaluate_model(label, model, X, y):
 model.fit(X,y)
 metric_train = AUC(model,X,y)
 metric_valida = cross_validation(model,X,y)
 
 label_.append(label)
 metric_train_.append(metric_train)
 metric_valida_.append(metric_valida)
 return pd.DataFrame({'Model': label_,
                'AUC Train': metric_train_,
                'AUC Cross-Valida': metric_valida_
                }).sort_values('AUC Cross-Valida')

In [31]:
evaluate_model(label='XGBoost', 
 model = xgb.XGBClassifier(), 
 X=X_train, y=y_train)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:mes: period[M]